In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [28]:
train = pd.read_csv("/Users/mingu/Desktop/CODING/Kaggle Project 1/DATASET/train.csv")
test = pd.read_csv("/Users/mingu/Desktop/CODING/Kaggle Project 1/DATASET/test.csv")

In [29]:
train.drop('Unnamed: 0', axis=1, inplace=True)
test.drop('Unnamed: 0', axis=1, inplace=True)

# preprocessing

In [30]:
### Species
train.drop('Species', axis = 1, inplace = True)
test.drop('Species', axis = 1, inplace = True)


### Farm.Name
train.drop('Farm.Name', axis = 1, inplace = True)
test.drop('Farm.Name', axis = 1, inplace = True)


#### Lot.Number
import re
import math

def map_country_of_origin_preprocessing(value):
    if pd.isna(value):
        return -2
    elif re.match(r'^\d+/\d+[A-Za-z]$', str(value)) or str(value).startswith('431')  or re.match(r'^\d{3}/\d{2}$', str(value)) or 'Lot' in str(value):
        return 0
    elif re.match(r'^3-\d{2}-\d{4}$', str(value)) or re.match(r'^\d{2}-\d{4}$', str(value)):
        return 1
    elif re.match(r'^11/\d+/\d+$', str(value)) or re.match(r'^11-\d+-\d+$', str(value)):
        return 2
    else:
        return -1  # 예외 처리

train['Mapped.Lot.Number.Country.of.Origin'] = train['Lot.Number'].apply(map_country_of_origin_preprocessing)
test['Mapped.Lot.Number.Country.of.Origin'] = test['Lot.Number'].apply(map_country_of_origin_preprocessing)

train.drop('Lot.Number', axis = 1, inplace = True)
test.drop('Lot.Number', axis = 1, inplace = True)


### Altitude

def to_meters(value):
    if pd.isnull(value):
        return np.nan
    
    if value.isdigit():
        return float(value)  # 이미 숫자인 경우 그대로 반환합니다.
    
    if isinstance(value, str):
        # 정규표현식을 사용하여 숫자를 추출합니다.
        numbers = re.findall(r'\d+\.?\d*', value)
        if len(numbers) == 1:
            number = float(numbers[0])
            # 단위에 따라 변환합니다.
            if 'ft' in value.lower():
                return number * 0.3048  # 피트(ft)를 미터(m)로 변환합니다.
            elif 'mts' in value.lower() or 'metros' in value.lower() or 'msn' in value.lower() or 'msnm' in value.lower():
                return number  # 미터(m)는 그대로 반환합니다.
            elif 'km' in value.lower():
                return number * 1000  # 킬로미터(km)를 미터(m)로 변환합니다.
            elif 'miles' in value.lower():
                return number * 1609.34  # 마일(miles)을 미터(m)로 변환합니다.
            elif 'pies' in value.lower():
                return number * 0.3048  # 피트(ft)를 미터(m)로 변환합니다.
            elif 'feet' in value.lower():
                return number * 0.3048  # 피트(ft)를 미터(m)로 변환합니다.
            else:
                return number
        elif len(numbers) == 2:
            # 숫자가 두 개인 경우에는 평균을 계산하여 반환합니다.
            avg_number = (float(numbers[0]) + float(numbers[1])) / 2
            return avg_number
    return np.nan

train['converted_altitude'] = train['Altitude'].apply(to_meters)
test['converted_altitude'] = test['Altitude'].apply(to_meters)

mean_altitude = train['converted_altitude'].mean()
train['converted_altitude'].fillna(mean_altitude, inplace=True)
test['converted_altitude'].fillna(mean_altitude, inplace=True)

train.drop('Altitude', axis = 1, inplace = True)
test.drop('Altitude', axis = 1, inplace = True)

### Bag.weight
# Bag.Weight 열의 값에 따라 kg로 변환하는 함수 정의
def convert_to_kg(value):
    # 만약 값이 비어있으면 그대로 반환
    if pd.isna(value):
        return value
    # 만약 값에 'kg'가 포함되어 있다면 이를 삭제한 후 반환
    elif 'kg' in str(value):
        return float(value.replace('kg', '').strip())
    # 만약 값이 'lbs'로 끝나면 lbs를 kg로 변환하여 반환
    elif str(value).endswith('lbs'):
        weight_in_lbs = float(value.replace('lbs', '').strip())
        weight_in_kg = weight_in_lbs * 0.453592  # 1 lbs = 0.453592 kg
        return weight_in_kg
    # 그 외의 경우는 단위가 없는 값으로 간주하여 kg로 반환
    else:
        return float(value)

# Bag.Weight 열에 적용하여 모든 값을 kg로 변환
train['Bag.Weight'] = train['Bag.Weight'].apply(convert_to_kg)
test['Bag.Weight'] = test['Bag.Weight'].apply(convert_to_kg)

### Harvest.Year
# Harvest.Year 열의 값에서 연도를 추출하여 반환하는 함수 정의
def extract_year(value):
    # 값이 비어있으면 nan 반환
    if pd.isna(value):
        return np.nan
    # 정규식을 사용하여 모든 연도 추출
    matches = re.finditer(r'\b(19|20)(\d{2})\b', str(value))
    # 추출된 연도가 없으면 nan 반환
    if not matches:
        return np.nan
    # 모든 매치의 연도를 리스트에 추가하여 반환
    years = []
    for match in matches:
        years.append(int(match.group()))
    return years

# Harvest.Year 열에 적용하여 모든 값을 연도로 변환
train['Harvest.Year'] = train['Harvest.Year'].apply(extract_year)
test['Harvest.Year'] = test['Harvest.Year'].apply(extract_year)

# 두 개 이상의 값이 있는 경우에는 두 값의 평균을 사용하는 함수 정의
def handle_multiple_years(value):
    # 만약 값이 nan이면 그대로 반환
    # 만약 값이 리스트 형태가 아니라면 그대로 반환
    if not isinstance(value, list):
        return value
    # 리스트의 길이가 1보다 작거나 같으면 그대로 반환
    if len(value) <= 1:
        return value[0] if len(value) == 1 else np.nan
    # 리스트의 길이가 2 이상이면 두 값의 평균을 계산하여 반환
    return np.mean(value)

# Harvest.Year 열에 적용하여 모든 값을 연도로 변환
train['Harvest.Year'] = train['Harvest.Year'].apply(handle_multiple_years)
test['Harvest.Year'] = test['Harvest.Year'].apply(handle_multiple_years)

max_harvest_year = train['Harvest.Year'].mode()[0]  # 최빈값 계산
train['Harvest.Year'].fillna(max_harvest_year, inplace=True)  # nan 값을 최빈값으로 대체
test['Harvest.Year'].fillna(max_harvest_year, inplace=True)

train['Harvest.Year'] = train['Harvest.Year']
test['Harvest.Year'] = test['Harvest.Year']

### Grading.Date 
# Grading.Date 열의 값을 파싱하여 년, 월, 일로 나누는 함수 정의
def parse_date(date_string):
    try:
        # 날짜 문자열을 datetime 객체로 변환
        date = pd.to_datetime(date_string)
        # 년, 월, 일 추출
        year = date.year
        month = date.month
        day = date.day
        return year, month, day
    except:
        return None, None, None

# Grading.Date 열에서 년, 월, 일 추출하여 새로운 열에 추가
train['Grading.Year'], train['Grading.Month'], train['Grading.Day'] = zip(*train['Grading.Date'].apply(parse_date))
test['Grading.Year'], test['Grading.Month'], test['Grading.Day'] = zip(*test['Grading.Date'].apply(parse_date))

# 월과 일에 대해 sin과 cos 함수를 적용하여 전처리하는 함수 정의
def preprocess_seasonality(month, day):
    # 월과 일을 0부터 1까지의 값으로 변환
    normalized_month = (month - 1) / 12  # 1월부터 12월까지를 0부터 1까지의 값으로 변환
    normalized_day = (day - 1) / 31  # 1일부터 31일까지를 0부터 1까지의 값으로 변환

    # sin과 cos 함수를 적용하여 전처리
    month_sin = np.sin(2 * np.pi * normalized_month)
    month_cos = np.cos(2 * np.pi * normalized_month)
    day_sin = np.sin(2 * np.pi * normalized_day)
    day_cos = np.cos(2 * np.pi * normalized_day)

    return month_sin, month_cos, day_sin, day_cos

# Grading.Date 열에서 월과 일 추출하여 전처리 적용
train['Grading.Month_Sin'], train['Grading.Month_Cos'], train['Grading.Day_Sin'], train['Grading.Day_Cos'] = zip(*train.apply(lambda x: preprocess_seasonality(x['Grading.Month'], x['Grading.Day']), axis=1))
test['Grading.Month_Sin'], test['Grading.Month_Cos'], test['Grading.Day_Sin'], test['Grading.Day_Cos'] = zip(*test.apply(lambda x: preprocess_seasonality(x['Grading.Month'], x['Grading.Day']), axis=1))

train.drop('Grading.Date', axis = 1, inplace = True)
test.drop('Grading.Date', axis = 1, inplace = True)

### Variety
from sklearn.preprocessing import LabelEncoder
# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

# train 데이터에서 Variety 열을 기반으로 라벨 인코딩 규칙을 학습하고 적용
train['Variety'] = label_encoder.fit_transform(train['Variety'])
# test 데이터에서 Variety 열을 라벨 인코딩 규칙을 적용 (train 데이터에서 학습된 규칙을 사용)
test['Variety'] = label_encoder.transform(test['Variety'])

# postprocessing

In [ ]:
###Farm.Name

farm_country_mapping = train[train['Farm.Name'].isin(['santa maria', 'several', 'various']) == False].groupby('Farm.Name')['Country.of.Origin'].first().to_dict()

# 후처리 함수 정의
def postprocess_country(df, farm_country_mapping):
    for index, row in df.iterrows():
        farm_name = row['Farm.Name']
        if farm_name in farm_country_mapping:
            df.at[index, 'Country.of.Origin'] = farm_country_mapping[farm_name]
    return df

# 후처리 적용
test = postprocess_country(test, farm_country_mapping)


### Lot.Number
import re

def map_country_of_origin_postprocessing(value):
    if re.match(r'^\d+/\d+[A-Za-z]$', str(value)) or str(value).startswith('431') or re.match(r'^\d{3}/\d{2}$', str(value)) or 'Lot' in str(value):
        return 0
    elif re.match(r'^3-\d{2}-\d{4}$', str(value)) or re.match(r'^\d{2}-\d{4}$', str(value)):
        return 1
    elif re.match(r'^11/\d+/\d+$', str(value)) or re.match(r'^11-\d+-\d+$', str(value)):
        return 2
    

# 후처리 함수 정의
def postprocess_country_of_origin(df):
    for index, row in df.iterrows():
        df.at[index, 'Country.of.Origin'] = map_country_of_origin_postprocessing(row['Country.of.Origin'])
    return df

# test_data에 후처리 적용
test = postprocess_country_of_origin(test)

### Altitude <- 1,2 혼동이 되는 경우가 있기 때문에 후처리 시 앞으로 배정
def map_country_based_on_altitude(country, altitude_text):
    if isinstance(altitude_text, str):  # 문자열인지 확인
        if 'msn' in altitude_text:
            if country not in [1, 2]:
                return 1
        elif 'psn' in altitude_text or 'ft' in altitude_text:
            return 2
    return country

test['Country.of.Origin'] = test.apply(lambda row: map_country_based_on_altitude(row['Country.of.Origin'], row['Altitude']), axis=1)

